In [20]:
import opendatasets as od

dataset_url = 'https://www.kaggle.com/datasets/dcaffo/myoemgdata'

od.download(dataset_url,)

Skipping, found downloaded files in ".\myoemgdata" (use force=True to force download)


In [21]:
import numpy as np

dataset_path = './myoemgdata/myo_ds_30l_10ol.npz'

data = np.load(dataset_path)
X, y = data['X'], data['y']

In [22]:
X

array([[[ 19,  26,  48, ...,  75,  28,  22],
        [ 20,  26,  49, ...,  73,  30,  23],
        [ 20,  23,  47, ...,  83,  34,  22],
        ...,
        [ 18,  22,  46, ...,  63,  23,  16],
        [ 20,  20,  37, ...,  57,  22,  15],
        [ 18,  23,  40, ...,  59,  23,  17]],

       [[ 19,  30,  41, ...,  75,  28,  19],
        [ 17,  25,  33, ...,  71,  27,  18],
        [ 18,  24,  33, ...,  81,  27,  17],
        ...,
        [ 18,  22,  67, ...,  78,  27,  23],
        [ 18,  21,  71, ...,  78,  31,  21],
        [ 18,  23,  67, ...,  69,  30,  20]],

       [[ 20,  21,  46, ...,  84,  37,  23],
        [ 22,  21,  44, ...,  86,  36,  23],
        [ 21,  21,  39, ...,  83,  34,  22],
        ...,
        [ 19,  24,  48, ...,  57,  27,  18],
        [ 19,  24,  50, ...,  64,  28,  18],
        [ 21,  24,  46, ...,  69,  31,  20]],

       ...,

       [[ 28, 140,  99, ...,  92,  24,  30],
        [ 24, 130,  96, ...,  94,  28,  31],
        [ 26, 133,  93, ..., 132,  35,  60

In [23]:
y

array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)

In [24]:
X.shape

(5595, 30, 8)

In [25]:
y.shape

(5595,)

In [26]:
X0 = X[y==0]
X0.shape

(2787, 30, 8)

In [27]:
X0_reshaped = X0.reshape(X0.shape[0]*X0.shape[1], X0.shape[2])
X0_reshaped

array([[ 19,  26,  48, ...,  75,  28,  22],
       [ 20,  26,  49, ...,  73,  30,  23],
       [ 20,  23,  47, ...,  83,  34,  22],
       ...,
       [ 23, 109,  65, ..., 162,  44,  60],
       [ 22, 109,  64, ..., 166,  45,  60],
       [ 21, 109,  62, ..., 140,  45,  61]], dtype=int16)

In [28]:
#X0_trimmed = X0_reshaped[:,:2]
#X0_trimmed

In [29]:
#X0_trimmed.shape

In [30]:
set(y)

{0, 1, 2, 7}

In [31]:
y_replaced = np.where(y==7, 3, y)
set(y_replaced)

{0, 1, 2, 3}

In [32]:
from pathlib import Path

path = Path('./data/kaggle_dataset')

path.mkdir(parents=True,exist_ok=True)

In [33]:
X0_path = path.joinpath('0_neutral.csv')
X0_path

WindowsPath('data/kaggle_dataset/0_neutral.csv')

In [34]:
import pandas as pd

channel_count = X.shape[2]
channels = [ f'emg{i}' for i in range(channel_count) ]
targets = [ 'neutral', 'flexion', 'extension', 'fist' ]

for i, target in enumerate(targets):
    Xi = X[y_replaced==i]
    Xi_reshaped = Xi.reshape(Xi.shape[0]*Xi.shape[1], Xi.shape[2])
#    Xi_trimmed = Xi_reshaped[:,:channel_count]

    Xi_path = path.joinpath(f'{i}_{target}.csv')
    df = pd.DataFrame(Xi_reshaped, columns=channels)
    df.to_csv(Xi_path, index=False)

In [35]:
pd.read_csv(X0_path)

,emg0,emg1,emg2,emg3,emg4,emg5,emg6,emg7
0,19,26,48,173,215,75,28,22
1,20,26,49,170,204,73,30,23
2,20,23,47,168,196,83,34,22
3,19,24,50,174,181,81,31,21
4,18,24,49,186,203,78,31,21
...,...,...,...,...,...,...,...,...
83605,23,111,71,108,319,174,40,33
83606,21,95,66,102,305,161,35,59
83607,23,109,65,112,318,162,44,60
83608,22,109,64,116,329,166,45,60


In [36]:
pd.read_csv(X0_path, usecols=['emg1', 'emg2'])

,emg1,emg2
0,26,48
1,26,49
2,23,47
3,24,50
4,24,49
...,...,...
83605,111,71
83606,95,66
83607,109,65
83608,109,64


In [37]:
from os import scandir

def get_data(dataset_name, channels):    
    path = f'./data/{dataset_name}'
    
    with scandir(path) as it:
        entries = []
        for entry in it:
            if not entry.name.startswith('.') and entry.name.endswith('.csv') and entry.is_file():
                entries.append(entry)
    
    records = [None] * len(entries)
    targets = [None] * len(entries)
    
    
    for entry in entries:
        i = int(entry.name[:entry.name.index('_')])
        targets[i] = entry.name[entry.name.index('_')+1:entry.name.index('.')]
        records[i] = pd.read_csv(entry.path, usecols=channels)
    return records, channels, targets

get_data('kaggle_dataset', ['emg0', 'emg1'])

([       emg0  emg1
  0        19    26
  1        20    26
  2        20    23
  3        19    24
  4        18    24
  ...     ...   ...
  83605    23   111
  83606    21    95
  83607    23   109
  83608    22   109
  83609    21   109
  
  [83610 rows x 2 columns],
         emg0  emg1
  0       123   205
  1       127   252
  2       138   261
  3       141   244
  4       171   296
  ...     ...   ...
  28075    27   220
  28076    27   190
  28077    28   183
  28078    29   184
  28079    27   173
  
  [28080 rows x 2 columns],
         emg0  emg1
  0        19    26
  1        17    23
  2        16    21
  3        18    21
  4        17    20
  ...     ...   ...
  28075    23    80
  28076    22    73
  28077    20    73
  28078    22    81
  28079    24    72
  
  [28080 rows x 2 columns],
         emg0  emg1
  0        18    21
  1        17    21
  2        19    24
  3        19    25
  4        17    28
  ...     ...   ...
  28075    29    68
  28076    37    71
  28077